Note:   
This notebook is meant to be open with Google Colab  
(Notebook này được thiết kế cho Google Colab)

In [ ]:
# Required installations
!pip install grpcio==1.63.0
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain-anthropic==0.1.20
!pip install langchain_milvus langchain-openai langchain-community
!pip install sentence-transformers
!npm install -g degit


In [ ]:
!npx degit STEAMforVietnam/genai-agents-for-teachers/agents agents --force
!npx degit STEAMforVietnam/genai-agents-for-teachers/data data --force
!npx degit STEAMforVietnam/genai-agents-for-teachers/prompts prompts --force
!npx degit STEAMforVietnam/genai-agents-for-teachers/tools tools --force
!npx degit STEAMforVietnam/genai-agents-for-teachers/utils utils --force
!npx degit STEAMforVietnam/genai-agents-for-teachers/outputs outputs --force

In [ ]:
# from google.colab import userdata
import os

## PHẦN 1: HƯỚNG DẪN VÀ SET CÁC BIẾN ĐỂ CHẠY

(Hiện tại kho dữ liệu support đang có nội dung của sách giáo khoa môn Toán, Sinh học, Vật lý lớp 10. Các thầy cô hãy lựa chọn thử 1 trong 3 nhé
Lưu Ý: Có lúc Agents sẽ làm việc tốt hơn nếu chúng ta không bỏ dấu cho subject. Ví dụ: subject = "Toan")


In [ ]:
#### CÁC THẦY CÔ CÓ THỂ THAY ĐỔI CÁC BIẾN DƯỚI ĐÂY
subject="Vat ly"

#### CÁC THẦY CÔ KHÔNG THAY ĐỔI CÁC BIẾN DƯỚI ĐÂY
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["DATABSE_PUBLIC_ENDPOINT"] = userdata.get("DATABSE_PUBLIC_ENDPOINT")
os.environ["DATABASE_API_KEY"] = userdata.get("DATABASE_API_KEY")
os.environ["OPENAI_MODEL_NAME"]="gpt-4o-mini"

## PHẦN 2: TẠO MỘT ĐỘT NGŨ NHÂN VIÊN (Crew)

### Phòng ban (Crew) 1: Dự Án Tạo Ma trận đề thi
Đội ngũ nhân viên này sẽ bao gồm 4 bạn nhân viên chính:
- Ngừoi Giám Sát phụ trách hướng dẫn, điều phối, và ra quyết định cho dự án
- Người Kiến Tạo phụ trách tác vụ tạo ma trận đề thi
- Người Kiểm Định phụ trách kiểm tra ma trận
- Người Thiết kế phụ trách tạo file HTML theo định dạng có sẵn

In [ ]:
from agents.matrix import MatrixCrew

matrix_crew = MatrixCrew()
result = matrix_crew.run(subject=subject)

### Phòng ban (Crew) 2: Dự Án Tạo đề thi
Đội ngũ nhân viên này sẽ bao gồm 4 bạn nhân viên chính:
- Ngừoi Giám Sát phụ trách hướng dẫn, điều phối, và ra quyết định cho dự án
- Người Kiến Tạo phụ trách tác vụ tạo ma trận đề thi
- Người Kiểm Định phụ trách kiểm tra ma trận
- Người Thiết kế phụ trách tạo file HTML theo định dạng có sẵn

Sau khi Đội 1 đã tạo ra được ma trận, đội 1 sẽ bàn giao kết quả đã tạo được (file `matrix.json`) cho đội 2. Đội 2 sẽ dùng kết quả này cho ra đề thi dứoi dạng html

In [ ]:
from agents.exam import ExamCrew

exam_crew = ExamCrew()
exam_result = exam_crew.run(subject=subject)


# LỜI KẾT:
Sau khi 2 đội ngũ của chúng ta đã thực hiện xong dự án của mình, chúng ta sẽ có
- 1 bộ ma trận đề thi dứoi dạng HTML
- 1 bộ đề thi dứoi dạng HTML